In [ ]:
# Importing necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
# Define the directory path containing the images
data_directory = "/Users/lukasiwan/NeueFische/Repositories/Hydroponics/data/train_data"
batch_size = 7
image_size = (50,50)
epoch_size = 250
fold_size = 3

In [ ]:
# Create the KFold object
kfold = KFold(n_splits=fold_size, shuffle=True, random_state=42)

In [ ]:
# Initialize lists to store the fold results
fold_train_loss = []
fold_train_accuracy = []
fold_val_loss = []
fold_val_accuracy = []

In [ ]:
# Perform k-fold cross-validation
fold = 1
for train_index, val_index in kfold.split(os.listdir(data_directory)):
    print(f"Fold {fold}:")

    # Load and preprocess the data for training set
    train_data = tf.keras.preprocessing.image_dataset_from_directory(
        data_directory,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        validation_split=0.2,
        subset="training"
    )

    # Load and preprocess the data for validation set
    val_data = tf.keras.preprocessing.image_dataset_from_directory(
        data_directory,
        batch_size=batch_size,
        image_size=image_size,
        shuffle=True,
        seed=42,
        validation_split=0.2,
        subset="validation"
    )

    # Define data augmentation
    data_augmentation = tf.keras.Sequential([
      preprocessing.Rescaling(1./255),
      preprocessing.Resizing(image_size[0], image_size[1]),
      preprocessing.RandomFlip("horizontal"),
      preprocessing.RandomRotation(0.3), 
      #preprocessing.RandomZoom(0.1)
    ])

    # Define the model architecture
    model = Sequential([
        data_augmentation,
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Dropout(0.25),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        Dropout(0.25),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_data, validation_data=val_data, epochs=epoch_size)

    # Store the fold results
    fold_train_loss.append(history.history['loss'])
    fold_train_accuracy.append(history.history['accuracy'])
    fold_val_loss.append(history.history['val_loss'])
    fold_val_accuracy.append(history.history['val_accuracy'])

    fold += 1

In [ ]:
# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(val_data)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Get training history
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Create line plots
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.title(f'Training and Validation Accuracy\n(Epochs: {epoch_size}, Batch Size: {batch_size}, Image Size: {image_size[0]}x{image_size[1]})')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title(f'Training and Validation Loss\n(Epochs: {epoch_size}, Batch Size: {batch_size}, Image Size: {image_size[0]}x{image_size[1]})')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()